# Elementy Inteligencji Obliczeniowej - Sieci Neuronowe


---

**Prowadzący:** REDACTED<br>
**Kontakt:** REDACTED<br>
**Materiały:** [Strona WWW](http://rrr.REDACTED.red)

---

## Uwaga

* **Aby wykonać polecenia należy najpierw przejść do trybu 'playground'. File -> Open in Playground Mode**
* Nowe funkcje Colab pozwalają na autouzupełnianie oraz czytanie dokumentacji

## Cel ćwiczeń:
- zapoznanie się z rekurencyjnymi sieciami neuronowymi,
- stworzenie modelu sieci z warstwami rekurencyjnymi dla zbioru danych MNIST,
- stworzenie własnych implementacji warstwami neuronowych

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, LSTM, LSTMCell, SimpleRNNCell
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta, RMSprop
from tensorflow.python.keras import backend as K


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()


x_train = x_train.astype('float32')  # shape: 60000, 28, 28
x_test = x_test.astype('float32')    # shape: 10000, 28, 28
x_train /= 255  # normalizacja wartości do przedziału [0, 1]
x_test /= 255

y_train = to_categorical(y_train, 10)  # zamiana etykiety na one-hot encoding; np. 2 -> [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_test = to_categorical(y_test, 10)

## Sieci rekurencyjne
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

https://www.tensorflow.org/guide/keras/rnn

https://www.tensorflow.org/guide/function

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/

Przykładowy model z warstwą rekurencyjną dla danych MNIST:

In [4]:
class RecurrentModel(Model):

    def __init__(self, num_classes=10):
        super(RecurrentModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.lstm_1 = LSTM(128, activation='relu')
        self.dense_1 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.lstm_1(inputs)
        return self.dense_1(x)

model = RecurrentModel(num_classes=10)

In [5]:
model.compile(optimizer=RMSprop(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 77s 39ms/step - loss: 0.5792 - accuracy: 0.8264
Epoch 2/2
1875/1875 [==============================] - 72s 38ms/step - loss: 0.1435 - accuracy: 0.9594


### Zadanie 1
Rozszerz model z powyższego przykładu o kolejną warstwę rekurencyjną przed gęstą warstwą wyjściową.

Standardowe sieci neuronowe generują jeden wynik na podstawie jednego inputu, natomiast sieci rekurencyjne przetwarzają dane sekwencyjnie, w każdym kroku łącząc wynik poprzedniego przetwarzania i aktualnego wejścia. Dlatego domyślnym wejściem sieci neuronowej jest tensor 3-wymiarowy ([batch_size,sequence_size,sample_size]).
Domyślnie warstwy rekurencyjne w Kerasie zwracają tylko wyniki przetwarzania ostatniego
kroku (otrzymują tensor 3-wymiarowy, zwracają tensor 2-wymiarowy). Jeśli chcesz zwrócić sekwencje wyników wszystkich kroków przetwarzania dla warstwy rekurencyjnej, musisz ustawić parametr return_sequences=True.


In [6]:
class ChewstoneModel(Model):
    def __init__(self, num_classes):
        super().__init__(name='ChewstoneModel')
        self.num_classes = num_classes
        self.lstm1 = LSTM(128, activation='relu', return_sequences=True)
        self.lstm2 = LSTM(128, activation='relu')
        self.d1    = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.lstm2(x)
        x = self.d1(x)
        return x

m = ChewstoneModel(num_classes=10)
m.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
m_f = m.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 164s 87ms/step - loss: 0.5015 - accuracy: 0.8492
Epoch 2/2
1875/1875 [==============================] - 162s 86ms/step - loss: 0.1391 - accuracy: 0.9636


### Zadanie 2 
Wykorzystując model z przykładu, napisz sieć rekurencyjną przy użyciu SimpleRNNCell.

Cell implementuje tylko operacje wykonywane przez warstwę
rekurencyjną dla jednego kroku. Warstwy rekurencyjne w każdym kroku
łączą wynik operacji poprzedniego kroku i aktualny input.
Wykorzystaj pętle for do wielokrotnego wywołania komórki SimpleRNNCell (liczba kroków to liczba elementów w sekwencji). Aby wywołać SimpleRNNCell dla pojedynczego wejścia i stanu należy użyć jej metody ```call``` analogicznie jak w przypadku własnych modeli (tzn. ```my_model(input)```). 



Wywołanie zainicjalizowanej komórki rekurencyjnej wymaga podania aktualnego inputu i **listy macierzy** (w dokumentacji jest błąd, że ma to być macierz) stanów ukrytych poprzedniego kroku (SimpleRNNCell ma jeden stan, LSTMCell w następnym zadaniu ma dwa stany).

Trzeba zainicjalizować ukryty stan warstwy wartościami początkowymi (można wykorzystać rozkład normalny - tf.random.normal).

In [7]:
class ChewstoneModel2(Model):
    def __init__(self, num_classes):
        super().__init__(name='ChewstoneModel2')
        self.num_classes = num_classes
        self.rnn1 = SimpleRNNCell(128, activation='relu')
        self.d1   = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        h = tf.random.normal([inputs.shape[0], self.rnn1.units])
        for i in range(inputs.shape[1]):
          x, h = self.rnn1(inputs[:,i,:], h)
        x = self.d1(x)
        return x

m2 = ChewstoneModel2(num_classes=10)
m2.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
m2_f = m2.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 16s 8ms/step - loss: 0.4937 - accuracy: 0.8450
Epoch 2/2
1875/1875 [==============================] - 14s 8ms/step - loss: 0.2150 - accuracy: 0.9383


### Zadanie 3
Zamień komórkę rekurencyjną z poprzedniego zadania na LSTMCell (LSTMCell ma dwa stany ukryte).

In [8]:
class ChewstoneModel3(Model):
    def __init__(self, num_classes):
        super().__init__(name='ChewstoneModel3')
        self.num_classes = num_classes
        self.ltsm1 = LSTMCell(128, activation='relu')
        self.d1    = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        dims = (inputs.shape[0], self.ltsm1.units)
        h = [tf.random.normal(dims), tf.random.normal(dims)]
        for i in range(inputs.shape[1]):
          x, h = self.ltsm1(inputs[:,i,:], h)
        x = self.d1(x)
        return x

m3 = ChewstoneModel3(num_classes=10)
m3.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
m3_f = m3.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 65s 33ms/step - loss: 0.6234 - accuracy: 0.8109
Epoch 2/2
1875/1875 [==============================] - 60s 32ms/step - loss: 0.1424 - accuracy: 0.9598


### Zadanie 4
Wykorzystując model z poprzedniego zadania, stwórz model sieci
neuronowej z własną implementacją prostej warstwy rekurencyjnej.
- w call zamień self.lstm_cell_layer(x) na wywołanie własnej metody np. self.cell(x)
- w konstruktorze modelu usuń inicjalizację komórki LSTM i zastąp ją inicjalizacją warstw potrzebnych do stworzenia własnej komórki rekurencyjnej,
- stwórz metodę cell() wykonującą operacje warstwy rekurencyjnej,
- prosta warstwa rekurencyjna konkatenuje poprzedni wyniki i aktualny input, a następnie przepuszcza ten połączony tensor przez warstwę gęstą (Dense).

In [9]:
class ChewstoneModel4(Model):
    def __init__(self, num_classes):
        super().__init__(name='ChewstoneModel4')
        self.num_classes = num_classes
        self.d1_r = Dense(128, activation='relu')
        self.d2   = Dense(num_classes, activation='softmax')

    def cell(self, x, h):
        return self.d1_r(K.concatenate([x, h]))

    def call(self, inputs):
        h = tf.random.normal((inputs.shape[0], self.d1_r.units))
        for i in range(inputs.shape[1]):
          h = self.cell(inputs[:,i,:], h)
        x = self.d2(h)
        return x

m4 = ChewstoneModel4(num_classes=10)
m4.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
m4_f = m4.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 17s 8ms/step - loss: 0.5514 - accuracy: 0.8205
Epoch 2/2
1875/1875 [==============================] - 15s 8ms/step - loss: 0.2372 - accuracy: 0.9315


### Zadanie 5

Na podstawie modelu z poprzedniego zadania stwórz model z własną implementacją warstwy LSTM. Dokładny i zrozumiały opis działania warstwy LSTM znajduje się na [stronie](http://colah.github.io/posts/2015-08-Understanding-LSTMs/).

In [10]:
class ChewstoneModel5(Model):
    def __init__(self, num_classes):
        super().__init__(name='ChewstoneModel5')
        self.num_classes = num_classes
        self.d1_i = Dense(128, activation='sigmoid')
        self.d2_f = Dense(128, activation='sigmoid')
        self.d3_o = Dense(128, activation='sigmoid')
        self.d4_c = Dense(128, activation='tanh')
        self.d5   = Dense(num_classes, activation='softmax')

    def cell(self, x, h):
        c_prev, h_prev = h
        hax = K.concatenate([h_prev, x])
        c_next = self.d1_i(hax) * self.d4_c(hax) + self.d2_f(hax) * c_prev
        h_next = self.d3_o(hax) * K.tanh(c_next)
        return h_next, (c_next, h_next)

    def call(self, inputs):
        shape = (inputs.shape[0], 128)
        h = [tf.random.normal(shape), tf.random.normal(shape)]
        for i in range(inputs.shape[1]):
          x, h = self.cell(inputs[:,i,:], h)
        x = self.d5(x)
        return x

m5 = ChewstoneModel5(num_classes=10)
m5.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])
m5_f = m5.fit(x_train, y_train, batch_size=32, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 47s 22ms/step - loss: 0.3459 - accuracy: 0.8900
Epoch 2/2
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1101 - accuracy: 0.9666
